<a href="https://colab.research.google.com/github/pieva/SimPy/blob/main/Cascade_Switches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Queue Simulation in Python
### Ref: https://www.grotto-networking.com/DiscreteEventPython.html

In [16]:
#!pip install simpy

import random
import simpy
import matplotlib.pyplot as plt

# Set a specific seed value to make the random selection replicable
seed_value = 42
random.seed(seed_value)

""" A very simple class that represents a packet.
    This packet will run through a queue at a switch output port.
"""
class Packet(object):
    # Constructor method (initializer) for the Packet class.
    # It is called when a new instance of the class is created
    def __init__(self, time, size, id, src="a", dst="z", flow_id=0):
        # Sets the attributes of the packet object to the values provided as the arguments
        self.time = time       # the time the packet arrives at the output queue.
        self.size = size       # the size of the packet in bytes
        self.id = id           # an identifier for the packet
        self.src = src         # identifiers for source
        self.dst = dst         # identifiers for destination
        self.flow_id = flow_id # identifier for a flow

    # String representation method for the Packet class.
    # It defines what string will be returned when you call repr(packet) for a Packet object
    def __repr__(self):
        return "id: {}, src: {}, time: {}, size: {}".\
            format(self.id, self.src, self.time, self.size)


""" Packet Generator with given inter-arrival time distribution.
    Set the "out" member variable to the entity to receive the packet.
"""
class PacketGenerator(object):
    # Constructor method: called when a new instance of the class is created,
    # it initializes the attributes of the packet generator object based on the provided arguments
    def __init__(self, env, id,  adist, sdist, initial_delay=0, finish=float("inf"), flow_id=0):
        self.id = id          # identifier for the packet generator
        self.env = env
        self.adist = adist    # successive inter-arrival times of the packets
        self.sdist = sdist    # successive sizes of the packets
        self.initial_delay = initial_delay # starts generation after an initial delay. Default = 0
        self.finish = finish  # stops generation at the finish time. Default is infinite
        self.out = None
        self.packets_sent = 0 # packets counter
        self.action = env.process(self.run())  # starts the run() method as a SimPy process
        self.flow_id = flow_id

    # It is the generator function executed as a SimPy process when the simulation environment runs
    def run(self):
        # This line yields control to the simulation environment for a duration of initial_delay time units,
        yield self.env.timeout(self.initial_delay)

        # Start a loop that continues packet generation until the simulation time reaches the finish time
        while self.env.now < self.finish:
            # Yields control to the simulation environment for a duration determined by the value returned from the adist() function.
            yield self.env.timeout(self.adist())

            self.packets_sent += 1 # increment the packets_sent attribute to track the total number of packets generated

            ''' Creates a new Packet object with
            - the current simulation time (self.env.now),
            - a size determined by the value returned from the sdist() function,
            - an identifier (self.packets_sent),
            - a source identifier (self.id),
            - a flow identifier (self.flow_id).'''
            packet_size = int(self.sdist())  # Convert the floating-point size to an integer
            p = Packet(self.env.now, packet_size, self.packets_sent, src=self.id, flow_id=self.flow_id)

            # This line sends the generated packet p to the entity represented by the out attribute,
            self.out.put(p)


""" Switch output port with a given rate and buffer size limit in bytes.
    Set the "out" member variable to the entity to receive the packet.
"""
class SwitchPort(object):
    # Constructor method
    def __init__(self, env, ps, rate, qlimit=None, limit_bytes=True, debug=False):
        self.store = simpy.Store(env)  # simpy.Store object will act as a buffer for packets waiting to be processed by the switch port.
        self.rate = rate               # the bit rate of the port
        self.env = env
        self.ps = ps # The packet sink object
        self.out = None                # the entity (e.g., another switch or a packet sink) to which the switch port sends packets
        self.packets_rec = 0           # total number of packets received
        self.packets_drop = 0          # total number of packets dropped
        self.qlimit = qlimit           # buffer size limit in bytes or packets for the queue (including items in service).
        self.limit_bytes = limit_bytes # If true, the queue limit will be based on bytes, packets if false
        self.byte_size = 0             # Current size of the queue in bytes
        self.debug = debug
        self.busy = 0                  # Used to track if a packet is currently being sent
        self.action = env.process(self.run())  # starts the run() method as a SimPy process

    # Behavior of the switch port
    def run(self):
        # Set the out attribute to the packet sink object
        self.out = self.ps
        while True:
            # Wait for a packet to be available in the store. When a packet is available, it is received and stored in the variable msg.
            msg = (yield self.store.get())

            # The switch port is currently processing a packet
            self.busy = 1

            # Adjust the size of the available queue
            self.byte_size -= msg.size

            # Yield control to the simulation environment and waits for a duration equal to the transmission time based on the given bit rate
            yield self.env.timeout(msg.size*8.0/self.rate)

            # Send the processed packet to the entity represented by the out attribute
            self.out.put(msg)

            # Switch port has finished processing the packet and is ready to receive the next one
            self.busy = 0

            # If in debug mode, print information about the processed packet msg.
            if self.debug:
                print(msg)

    # This method allows external entities to put (send) a packet into the switch port
    def put(self, pkt):
        # Track the total number of packets received by the switch port
        self.packets_rec += 1

        # Calculate the temporary byte count of the queue
        tmp_byte_count = self.byte_size + pkt.size

        # Decide whether to put the packet into the queue or drop it
        if self.qlimit is None: # no buffer limit
            self.byte_size = tmp_byte_count
            return self.store.put(pkt)
        if self.limit_bytes and tmp_byte_count >= self.qlimit: # drop counting bytes
            self.packets_drop += 1
            return
        elif not self.limit_bytes and len(self.store.items) >= self.qlimit-1: # drop counting packets
            self.packets_drop += 1
        else:
            self.byte_size = tmp_byte_count # process the packet
            return self.store.put(pkt)


""" Receives packets and collects delay information into the
    waits list. You can then use this list to look at delay statistics.
"""
class PacketSink(object):
    # Constructor method
    def __init__(self, env, rec_arrivals=False, absolute_arrivals=False, rec_waits=True, debug=False, selector=None):
        self.store = simpy.Store(env) # buffer to hold the incoming packets
        self.env = env
        self.rec_waits = rec_waits # If it is True, the waiting time for each packet will be recorded
        self.rec_arrivals = rec_arrivals # If it is True, the arrival times will be recorded
        self.absolute_arrivals = absolute_arrivals # If it is True, the absolute arrival times will be recorded, otherwise the time between consecutive arrivals
        self.waits = []      # empty list to be used to store the waiting time
        self.arrivals = []   # empty list to be used to store arrival times
        self.debug = debug   # if true then the contents of each packet will be printed as it is received
        self.packets_rec = 0 # total number of packets received
        self.bytes_rec = 0   # total size of all packets received
        self.selector = selector # If the selector function returns False, no information is recorded
        self.last_arrival = 0.0  # It keeps track of the time of the last packet arrival to calculate inter-arrival times.
        self.out = None

        # Set the out attribute to the SwitchPort object
        self.out = None

    # This method allows external entities to put a packet into the packet sink.
    def put(self, pkt):
        if not self.selector or self.selector(pkt):
            now = self.env.now
            #  If rec_waits is True, append the waiting time to the waits list
            if self.rec_waits:
                self.waits.append(self.env.now - pkt.time)

            # the absolute or inter-arrival time is appended to the arrivals
            if self.rec_arrivals:
                if self.absolute_arrivals:
                    self.arrivals.append(now)
                else:
                    self.arrivals.append(now - self.last_arrival)
                self.last_arrival = now

            # track the total number of packets received
            self.packets_rec += 1

            # track the total size of all packets received
            self.bytes_rec += pkt.size

            # Send the packet to the entity represented by the out attribute
            if self.out is not None:
                self.out.put(pkt)  # Add this line to propagate the packet to the output entity

            if self.debug:
                print(pkt)

""" A monitor for an SwitchPort. Looks at the number of items in the SwitchPort
    in service + in the queue and records that info in the sizes[] list. The
    monitor looks at the port at time intervals given by the distribution dist.
"""
class PortMonitor(object):
    # constructor method
    def __init__(self, env, port, dist, count_bytes=False):
        self.port = port # the switch port object to be monitored.
        self.env = env
        self.dist = dist # returns the successive inter-arrival times of the packets to schedule the times to checks the queue size
        self.count_bytes = count_bytes
        self.sizes = [] # list to store the number of bytes/packets in the output port at different time points
        self.action = env.process(self.run())

    # periodically check and record the queue size in the switch output port
    def run(self):
        while True:
            yield self.env.timeout(self.dist())
            if self.count_bytes:
                total = self.port.byte_size
            else:
                total = len(self.port.store.items) + self.port.busy
            self.sizes.append(total)


# Set up arrival and packet size distributions
adist = functools.partial(random.expovariate, 0.5)   # generator inter-arrival times
sdist = functools.partial(random.expovariate, 0.01)  # generator mean size 100 bytes
port_rate = 1000.0 # switch port rate
qlimit=10000       # switch buffer size
samp_dist = functools.partial(random.expovariate, 1.0) # times at which the monitor checks the queue size

# Create the SimPy environment
env = simpy.Environment()

# Create the network nodes
pg = PacketGenerator(env, "Gen1", adist, sdist)
ps_1 = PacketSink(env, rec_arrivals=True, rec_waits=True, debug=False)
ps_2 = PacketSink(env, rec_arrivals=True, rec_waits=True, debug=False)
switch_1 = SwitchPort(env, ps_1, port_rate, qlimit)  # Pass ps_1 as the second argument
switch_2 = SwitchPort(env, ps_2, port_rate, qlimit)  # Pass ps_2 as the second argument

# Wire network nodes together
pg.out = switch_1
switch_1.out = ps_1
ps_1.out = switch_2
switch_2.out = ps_2

# Using PortMonitors to track queue sizes over time for both switches
pm_1 = PortMonitor(env, switch_1, samp_dist, count_bytes=False)
pm_2 = PortMonitor(env, switch_2, samp_dist, count_bytes=False)

# Run the simulation
env.run(until=10)

# Print the table for both switches
print("Switch 1:")
print("Packet ID\tQueue Size\tArrival Time\tWaiting Time\tDeparture Time")
for packet_id, (queue_size, arrival_time, waiting_time) in enumerate(zip(pm_1.sizes, ps_1.arrivals, ps_1.waits), start=1):
    departure_time = arrival_time + waiting_time
    print(f"{packet_id}\t\t{queue_size}\t\t{arrival_time:.6f}\t\t{waiting_time:.6f}\t\t{departure_time:.6f}")

print("\nSwitch 2:")
print("Packet ID\tQueue Size\tArrival Time\tWaiting Time\tDeparture Time")
for packet_id, (queue_size, arrival_time, waiting_time) in enumerate(zip(pm_2.sizes, ps_2.arrivals, ps_2.waits), start=1):
    departure_time = arrival_time + waiting_time
    print(f"{packet_id}\t\t{queue_size}\t\t{arrival_time:.6f}\t\t{waiting_time:.6f}\t\t{departure_time:.6f}")


Switch 1:
Packet ID	Queue Size	Arrival Time	Waiting Time	Departure Time
1		1		1.539248		0.896000		2.435248
2		1		3.886576		0.328000		4.214576
3		0		2.529973		2.520000		5.049973
4		0		1.312000		3.011262		4.323262

Switch 2:
Packet ID	Queue Size	Arrival Time	Waiting Time	Departure Time
1		0		2.435248		1.792000		4.227248
2		0		3.318576		0.656000		3.974576
